In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import yaml

from typing import Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

import optuna
from optuna.trial import TrialState

%load_ext kedro.ipython
device = torch.device('mps')

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [20]:
mox_bin = catalog.load("mox_bin")
# mox_bin.tail()
lstm_params = catalog.load("params:complete_lstm_model.model_options")
model_input_table = catalog.load("model_input_table")

[01/23/24 11:35:43] INFO     Loading data from 'mox_bin' (ParquetDataset)...                    ]8;id=141601;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=180187;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[01/23/24 11:35:44] INFO     Loading data from 'params:complete_lstm_model.model_options'       ]8;id=662231;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=435865;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from 'model_input_table' (ParquetDataset)...          ]8;id=915535;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=839875;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [9]:
print(lstm_params)
print(lstm_params['num_epochs'])

{'num_classes': 1, 'num_epochs': 100, 'batch_size': 32, 'learning_rate': 0.001, 'test_size': 0.2, 'random_state': 3, 'input_size': 1, 'sequence_length': 1500, 'hidden_size': 128, 'num_layers': 4, 'val_size': 0.2}
100


In [17]:

# Hyper-parameters 

num_classes = lstm_params['num_classes']
num_epochs = lstm_params['num_epochs']
batch_size = lstm_params['batch_size']
learning_rate = lstm_params['learning_rate']

"""

Each feature as a time step in your sequence, you could set sequence_length to 150 and input_size to 1.
This would mean you are feeding in sequences of length 150, with each time step in the sequence having 1 feature.

"""

input_size = lstm_params['input_size']
sequence_length = lstm_params['sequence_length'] # the window it trains with can be selected
hidden_size = lstm_params['hidden_size']
num_layers = lstm_params['num_layers']
random_state = lstm_params['random_state']

batch_size = lstm_params['batch_size']  # You can adjust the batch size according to your needs

test_size = lstm_params['test_size']

In [21]:
def split_data(model_input_table: pd.DataFrame) -> torch.tensor:
    # print(f"Test size: {parameters['test_size']}, type: {type(parameters['test_size'])}")
    # print(f"Random state: {parameters['random_state']}, type: {type(parameters['random_state'])}")

    # Split data into features and target
    X = model_input_table[model_input_table.columns[:-1]].values  # Assuming last column is the target
    y = model_input_table[model_input_table.columns[-1]].values
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    # Initialize StandardScaler
    scaler = StandardScaler()
    # Fit on training data
    scaler.fit(X_train)
    # Transform both training and testing data
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Ensure y_train and y_test are in the correct format
    if isinstance(y_train, pd.Series):
        y_train = y_train.values
    if isinstance(y_test, pd.Series):
        y_test = y_test.values
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32))
    y_train_tensor = torch.tensor(y_train.astype(np.float32))
    X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32))
    y_test_tensor = torch.tensor(y_test.astype(np.float32))

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

# create X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor from split_data(df)
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = split_data(model_input_table)
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Initialize DataLoaders

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Now, train_loader and test_loader can be used in your training loop

In [ ]:
# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 10)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


In [ ]:
# turn the block below into a function
def train_model (data: DataLoader)->():
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for i, (bins, target) in enumerate(train_loader):  
            bins = bins.reshape(-1, sequence_length, input_size).to(device)
            target = target.to(device)
        
        # Forward pass
        outputs = model(bins)
        # Example of reshaping/squeezing if applicable
        outputs = outputs.squeeze()  # Removes dimensions of size 1
        outputs = outputs[:64]  # Adjust if you need to slice the outputs

        target = target.unsqueeze(1).to(device)  # Add an extra dimension to match outputs
        loss = criterion(outputs, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

    # Calculate RMSE at the end of each epoch
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Don't calculate gradients
            total_loss = 0
            count = 0
            for bins, target in test_loader:  # Replace with your validation loader
                bins = bins.reshape(-1, sequence_length, input_size).to(device)
                target = target.unsqueeze(1).to(device)  # Add an extra dimension to match outputs
                outputs = model(bins)
                loss = criterion(outputs, target)
                total_loss += loss.item()
                count += 1
            rmse = np.sqrt(total_loss / count)
            print(f'Epoch [{epoch+1}/{num_epochs}], RMSE on validation data: {rmse}')
        model.train()  # Set the model back to training mode
    # Save the model after training
    lstm_model = torch.save(model.state_dict())
    return lstm_model